In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
with open('../config.json') as file:
    app_data = json.load(file)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials( \
                     client_id=app_data['client_id'], \
                     client_secret=app_data['client_secret']))

In [3]:
all_years = pd.read_csv('../data/Billboard_Audio_Features.csv')

In [4]:
all_genres = pd.DataFrame()
for rank_val, rank in all_years.groupby('Rank'):
    audio_features = pd.DataFrame (sp.artist(tracks=rank['ID']))
    rank_and_features = rank.merge(audio_features, left_on='ID', right_on='id', how='inner')
    rank_and_features = rank_and_features.drop(columns=['track_href','type','uri','analysis_url','id'])
    all_audio_features = all_audio_features.append(rank_and_features)

In [5]:
for a, song in all_audio_features.groupby(['Rank','Year']):
    if len(song) > 1:
        print(a)

# all_audio_features = all_audio_features.drop_duplicates(keep = 'first')

(100.0, 1969.0)


In [6]:
all_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6069 entries, 0 to 61
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Album ID          6069 non-null   object 
 1   Artist IDs        6069 non-null   object 
 2   Artist(s)         6069 non-null   object 
 3   ID                6069 non-null   object 
 4   Popularity        6069 non-null   float64
 5   Query             6069 non-null   object 
 6   Rank              6069 non-null   float64
 7   Release Date      6069 non-null   object 
 8   Song              6069 non-null   object 
 9   Year              6069 non-null   float64
 10  danceability      6069 non-null   float64
 11  energy            6069 non-null   float64
 12  key               6069 non-null   int64  
 13  loudness          6069 non-null   float64
 14  mode              6069 non-null   int64  
 15  speechiness       6069 non-null   float64
 16  acousticness      6069 non-null   float64
 1

In [8]:
all_audio_features.to_csv('../data/Billboard_Audio_Features.csv', index=False)